In [2]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import joblib


Mounted at /content/drive


In [3]:
BASE_PATH = "/content/drive/MyDrive/IIT_Roorkee_Project/data/"

X_train = pd.read_csv(BASE_PATH + "X_train.csv")
X_val   = pd.read_csv(BASE_PATH + "X_val.csv")
y_train = pd.read_csv(BASE_PATH + "y_train.csv").values.ravel()
y_val   = pd.read_csv(BASE_PATH + "y_val.csv").values.ravel()

print(X_train.shape, X_val.shape)


(12967, 14) (3242, 14)


In [4]:
# Drop ID (never a feature)
X_train = X_train.drop(columns=['id'])
X_val   = X_val.drop(columns=['id'])

# Fresh scaler for shallow model
scaler_shallow = StandardScaler()

X_train_scaled = scaler_shallow.fit_transform(X_train)
X_val_scaled   = scaler_shallow.transform(X_val)

# Save this scaler separately
joblib.dump(scaler_shallow, BASE_PATH + "tabular_scaler_shallow.pkl")


['/content/drive/MyDrive/IIT_Roorkee_Project/data/tabular_scaler_shallow.pkl']

In [5]:
X_train_t = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.float32).view(-1,1)

X_val_t = torch.tensor(X_val_scaled, dtype=torch.float32)
y_val_t = torch.tensor(y_val, dtype=torch.float32).view(-1,1)


In [6]:
train_ds = TensorDataset(X_train_t, y_train_t)
val_ds   = TensorDataset(X_val_t, y_val_t)

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=64, shuffle=False)


In [7]:
class ShallowMLP(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 32),   # 🔥 small capacity
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.net(x)


In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = ShallowMLP(input_dim=X_train.shape[1]).to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


In [9]:
EPOCHS = 50

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0

    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)

        optimizer.zero_grad()
        preds = model(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{EPOCHS}, Train MSE: {train_loss/len(train_loader):.4f}")


Epoch 10/50, Train MSE: 1.0211
Epoch 20/50, Train MSE: 0.1624
Epoch 30/50, Train MSE: 0.0661
Epoch 40/50, Train MSE: 0.0476
Epoch 50/50, Train MSE: 0.0416


In [10]:
model.eval()
val_preds, val_true = [], []

with torch.no_grad():
    for xb, yb in val_loader:
        xb = xb.to(device)
        preds = model(xb).cpu().numpy()

        val_preds.extend(preds)
        val_true.extend(yb.numpy())

val_preds = np.array(val_preds).ravel()
val_true  = np.array(val_true).ravel()

rmse = np.sqrt(mean_squared_error(val_true, val_preds))
r2   = r2_score(val_true, val_preds)

print(f"Shallow MLP RMSE: {rmse:.4f}")
print(f"Shallow MLP R²  : {r2:.4f}")


Shallow MLP RMSE: 0.2828
Shallow MLP R²  : 0.7103


In [11]:
torch.save(model.state_dict(), BASE_PATH + "mlp_tabular_shallow.pth")
